In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [2]:
data = pd.read_csv('all_data.csv', index_col = 0)

In [3]:
#grouping income and broadband usage
data['Median Income Group']= pd.cut(data['Median Income'], 
                                        bins=[39000,50000,65000,80000,120000], 
                                        labels=['Very Low','Low','Moderate','High'])
data['Broadband Speed Category'] = pd.cut(data['BROADBAND USAGE'], 
                                        bins=[0,20,40,60,80], 
                                        labels=['Very Low','Low', 'Moderate', 'High'])

In [4]:
#importing and formatting the cleaned data
v = data.columns
w = v.str.lstrip()
w = w.str.replace('_', '')
rename = {v:w for (v,w) in zip(v,w)}
data.rename(rename, axis = 1, inplace = True)
data['BROADBAND USAGE'] = data['BROADBAND USAGE']*100
data.head()

,County,number18,%adv18,%prof18,%bas18,%belbas18,number22,%adv22,%prof22,%bas22,%belbas22,10 to 14 year olds enrolled in school,15 to 17 year olds enrolled in school,5 to 9 year olds enrolled in school,BROADBAND USAGE,Median Income,Median Income Group,Broadband Speed Category
0,ADAMS,83.0,17.60,38.3,28.6,9.60,112.0,15.90,36.55,30.05,17.05,5305.0,4161.0,5034.0,24.50,67253,Moderate,Very Low
1,ALLEGHENY,61.0,17.15,32.9,24.5,9.85,58.0,13.25,29.30,25.40,15.80,59856.0,38359.0,61209.0,47.90,61043,Low,Very Low
2,ARMSTRONG,85.5,19.15,40.9,27.4,7.95,72.5,15.65,37.90,29.00,12.10,3073.0,2258.0,3812.0,24.00,51410,Low,Very Low
3,BEAVER,79.0,14.80,37.7,27.0,12.20,77.0,11.40,33.30,29.50,18.00,9066.0,5852.0,7693.0,30.75,57807,Low,Very Low
4,BEDFORD,68.0,14.30,38.1,31.4,9.20,69.0,14.10,32.90,29.20,15.20,NaN,NaN,NaN,12.25,50509,Low,Very Low


In [5]:
df = data.groupby(by = 'Median Income Group',as_index = False)[['%belbas18','%belbas22']].mean()
df
#,'Below Basic Change'

C:\Users\HP\AppData\Local\Temp\ipykernel_30200\3093382484.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = data.groupby(by = 'Median Income Group',as_index = False)[['%belbas18','%belbas22']].mean()


,Median Income Group,%belbas18,%belbas22
0,Very Low,10.984375,17.328125
1,Low,10.988750,17.388750
2,Moderate,9.631250,15.031250
3,High,5.750000,9.783333


In [6]:
id_vars = 'Median Income Group'
var_name = ['%belbas18','%belbas22']
value_name = "Counts"

temp_melted = pd.melt(df, id_vars, 
                var_name, 
                value_name)
temp_melted.rename({'Median Income Group':'MedianIncomeGroup'},inplace=True,axis=1)
temp_melted

,MedianIncomeGroup,Counts,value
0,Very Low,%belbas18,10.984375
1,Low,%belbas18,10.988750
2,Moderate,%belbas18,9.631250
3,High,%belbas18,5.750000
4,Very Low,%belbas22,17.328125
5,Low,%belbas22,17.388750
6,Moderate,%belbas22,15.031250
7,High,%belbas22,9.783333


In [8]:
import plotly.graph_objects as go
from plotly import data

import pandas as pd

#temp_melted = data.gapminder()

countries = (
    temp_melted["MedianIncomeGroup"]
    .unique()
)

data = {"line_x": [], "line_y": [], "% in Below Basic Category in 2018": [], "% in Below Basic Category in 2022": [], "colors": [], "Counts": [], "countries": []}

for x in countries:
    data["% in Below Basic Category in 2018"].extend([temp_melted.loc[(temp_melted.Counts == '%belbas18') & (temp_melted.MedianIncomeGroup == x)]["value"].values[0]])
    data["% in Below Basic Category in 2022"].extend([temp_melted.loc[(temp_melted.Counts == '%belbas22') & (temp_melted.MedianIncomeGroup == x)]["value"].values[0]])
    data["line_x"].extend(
        [
            temp_melted.loc[(temp_melted.Counts == '%belbas18') & (temp_melted.MedianIncomeGroup == x)]["value"].values[0],
            temp_melted.loc[(temp_melted.Counts == '%belbas22') & (temp_melted.MedianIncomeGroup == x)]["value"].values[0],
            None,
        ]
    )
    data["line_y"].extend([x, x, None]),

fig = go.Figure(
    data=[
        go.Scatter(
            x=data["line_x"],
            y=data["line_y"],
            mode="lines",
            showlegend=False,
            marker=dict(
                color="black"
            )
        ),
        go.Scatter(
            x=data["% in Below Basic Category in 2018"],
            y=countries,
            mode="markers",
            name="2018",
            marker=dict(
                color='rgba(38, 24, 74, 0.8)',
                size=10
            )
# 'rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)','rgba(122, 120, 168, 0.8)'           
        ),
        go.Scatter(
            x=data["% in Below Basic Category in 2022"],
            y=countries,
            mode="markers",
            name="2022",
            marker=dict(
                color='rgba(122, 120, 168, 0.8)' ,
                size=10
            )   
        ),
    ]
)

fig.update_layout(yaxis_title="Income Group",xaxis_title="% Students in Below-Basic Score Category",
    font=dict(
        family="Arial",
        size=10,
        color="black"
    ),
    autosize=False,
    width=500,
    height=300,
    title="% of Students in Below-Basic Score Category in Each Income Group",
    #height=300,
    legend_itemclick=False,
    paper_bgcolor='white',
    plot_bgcolor='white',
   xaxis=dict(showgrid=False),
   yaxis=dict(showgrid=False)
)

fig.show()

References
1. https://stackoverflow.com/questions/46173419/change-bar-color-according-to-hue-name
2. https://www.geeksforgeeks.org/bar-chart-using-plotly-in-python/
3. https://stackoverflow.com/questions/42535785/python-plotly-how-to-remove-background-horizontal-line
4. https://stackoverflow.com/questions/56712486/how-to-hide-legend-with-plotly-express-and-plotly
5. https://python-charts.com/ranking/bar-chart-plotly/
6. PreAttentiveFeatures_P1_inclassEx_Grace_Sharon_Sami
7. https://stackoverflow.com/questions/71052703/how-to-plot-a-horizontal-stacked-bar-plot-using-plotly-python
8. https://plotly.com/python/horizontal-bar-charts/
9. https://www.flatuicolorpicker.com/red-rgba-color-model/
10. https://plotly.com/python/dumbbell-plots/
11. https://stackoverflow.com/questions/47897607/groupby-a-dataframe-into-a-new-dataframe-with-arange-as-index